# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902070


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/32 [00:05<01:22,  2.74s/it]

Rendering models:  12%|█▎        | 4/32 [00:05<00:54,  1.94s/it]

Rendering models:  19%|█▉        | 6/32 [00:05<00:35,  1.38s/it]

Rendering models:  22%|██▏       | 7/32 [00:07<00:36,  1.45s/it]

...

Rendering models:  88%|████████▊ | 28/32 [00:09<00:00,  7.61it/s]

Rendering models:  91%|█████████ | 29/32 [00:09<00:00,  7.03it/s]

Rendering models:  94%|█████████▍| 30/32 [00:09<00:00,  6.52it/s]

Rendering models: 100%|██████████| 32/32 [00:10<00:00,  7.70it/s]

equidad1                              0.089831
kayleebug2017                         0.002092
JessieDawn                            0.207486
not-logged-in-621b62a5f3840545c538    0.003850
Thedunkmasta                          0.001430
BirdLeaf                              0.000454
mlang2017                             0.032239
EastonH                               0.003268
not-logged-in-1bfb5ea67e2d65ad84ff    0.001452
Linda_J._Berkel                       0.000193
next_ghost                            0.000444
Vbgil13                               0.000628
Paddy_van_CH                          0.000171
Paddy_van_CH                          0.000386
JEBala                                0.000284
maevepower                            0.002388
lsautter                              0.000206
not-logged-in-7c0e3b1ae8dc7fbb6b66    0.001140
ElisabethB                            0.000154
kawaboy                               0.007960
jamesrc72                             0.037028
Samuel_Liuwe 

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())